In [ ]:
from connection_script import get_engine
import pandas as pd
import numpy as np
from sqlalchemy import text, inspect, MetaData, Table, Column, Integer, String, Boolean, Date
from datetime import datetime, timedelta
from pandas.errors import OutOfBoundsDatetime

# Obtener las conexiones a las bases de datos
engine_bodega = get_engine('bodega_datos')
engine_etl = get_engine('etl_rapidos_furiosos')

# Función para crear y llenar la tabla 'fechas' si no existe o si está vacía
def crear_y_llenar_tabla_fechas(engine):
    # Inspector para verificar si la tabla ya existe
    inspector = inspect(engine)
    if not inspector.has_table("fechas"):
        metadata = MetaData()
        
        # Definir estructura de la tabla fechas
        fechas = Table(
            "fechas", metadata,
            Column("id_fecha", Integer, primary_key=True),
            Column("fecha", Date, nullable=False),
            Column("dia", Integer, nullable=False),
            Column("mes", Integer, nullable=False),
            Column("año", Integer, nullable=False),
            Column("nombre_dia", String(10), nullable=False),
            Column("nombre_mes", String(15), nullable=False),
            Column("es_fin_de_semana", Boolean, nullable=False)
        )
        
        # Crear la tabla en la base de datos
        metadata.create_all(engine)
        print("Tabla 'fechas' creada exitosamente.")
        
    # Verificar si la tabla 'fechas' está vacía
    with engine.connect() as conn:
        result = conn.execute(text("SELECT COUNT(*) FROM fechas"))
        count = result.scalar()
        
    # Llenar la tabla solo si está vacía
    if count == 0:
        print("Llenando la tabla 'fechas' con fechas para los próximos 3 años.")
        
        # Rellenar la tabla con fechas desde 2023 hasta tres años en adelante
        fechas = []
        start_date = datetime(2023, 1, 1)
        end_date = datetime(2026, 1, 1)
        current_date = start_date
        id_fecha = 1
        
        while current_date < end_date:
            dia = current_date.day
            mes = current_date.month
            año = current_date.year
            nombre_dia = current_date.strftime("%A")  # Ej. "Monday"
            nombre_mes = current_date.strftime("%B")  # Ej. "January"
            es_fin_de_semana = current_date.weekday() >= 5  # Sábado(5) o Domingo(6)
            
            fechas.append({
                "id_fecha": id_fecha,
                "fecha": current_date,
                "dia": dia,
                "mes": mes,
                "año": año,
                "nombre_dia": nombre_dia,
                "nombre_mes": nombre_mes,
                "es_fin_de_semana": es_fin_de_semana
            })
            
            id_fecha += 1
            current_date += timedelta(days=1)
        
        # Convertir a DataFrame y cargar en la tabla 'fechas'
        df_fechas = pd.DataFrame(fechas)
        with engine.connect() as conn:
            df_fechas.to_sql('fechas', conn, if_exists='append', index=False)
        
        print("Tabla 'fechas' llenada exitosamente con fechas para los próximos 3 años.")
    else:
        print("La tabla 'fechas' ya tiene datos y no se insertaron nuevas filas.")

# Ejecutar el proceso de creación y carga de 'fechas'
crear_y_llenar_tabla_fechas(engine_etl)


Llenando la tabla 'fechas2' con fechas para los próximos 3 años.
Tabla 'fechas2' llenada exitosamente con fechas para los próximos 3 años.


In [5]:
# Esta celda se ejecuta primero para obtener la conexión
from connection_script import get_engine

# Obtener el motor de conexión para la base de datos 'etl_rapidos_furiosos'
engine_etl = get_engine('etl_rapidos_furiosos')


In [6]:
# Esta celda llama a la función que creará y llenará la tabla 'fechas2'
crear_y_llenar_tabla_fechas(engine_etl)


La tabla 'fechas2' ya tiene datos y no se insertaron nuevas filas.
